In [28]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [22]:
ratingsdf=pd.read_csv('ml-latest-small/ratings.csv',usecols=["userId",
                                                              'movieId',
                                                             'rating'])

In [23]:
ratingsdf.head()
ratingsdf.shape

(100836, 3)

In [27]:
userids=ratingsdf['userId'].unique()
movieIds=ratingsdf['movieId'].unique()

In [50]:
def SimPearson(df , User1 , User2 , min_common_items = 10):
    # GET MOVIES OF USER1
    mov_u1 = df[df['userId'] == User1 ]
    # GET MOVIES OF USER2
    mov_u2 = df[df['userId'] == User2 ]
    # FIND SHARED FILMS
    rep = pd.merge( mov_u1 , mov_u2 , on = 'movieId')
    if len(rep)==0:
        return 0
    if(len(rep) < min_common_items):
        return 0
    return pearsonr(rep['rating_x'], rep['rating_y']) [0]

In [53]:
SimPearson(ratingsdf,1,5)

0.26874902634577136

In [69]:
class CollaborativeFiltering:
    """ CF using a custom sim(u,u ’). """
    def __init__(self , df, similarity = SimPearson):
        """ Constructor """
        self. sim_method = similarity
        self.df = df
        self.sim = pd. DataFrame(
        np.sum ([0]), columns = df.userId.unique(),
        index = df.userId.unique())
    def fit(self):
        """ Prepare data structures for estimation.
        Similarity matrix for users """
        allUsers = set(self.df['userId'])
        self.sim = {}
        for person1 in allUsers:
            self.sim. setdefault( person1 , {})
            a = self.df[
            self.df['userId'] == person1][['movieId']]
            data_reduced = pd. merge(self.df,a,
            on = 'movieId')
            for person2 in allUsers:
                # Avoid our -self
                if person1 <= person2: continue
                self.sim. setdefault( person2 , {})
#                 if(self.sim[person2].has_key(person1)):
#                     continue # since symmetric matrix
                sim = self. sim_method( data_reduced ,
                person1 ,
                person2)
                if(sim < 0):
                    self.sim[person1][person2] = 0
                    self.sim[person2][person1] = 0
                else :
                    self.sim[person1][person2] = sim
                    self.sim[person2][person1] = sim
    def predict(self , user_id , movie_id):
        totals = {}
        users = self.df[self.df['movieId'] == movie_id]
        rating_num , rating_den = 0.0, 0.0
        allUsers = set(users['userId'])
        for other in allUsers:
            if user_id == other: continue
            rating_num +=self.sim[user_id][other] * float (users[users
            ['userId'] == other]['rating'])
            rating_den += self.sim[user_id][other]
        if rating_den == 0:
            if self.df.rating [self.df['movieId'] ==
            movie_id]. mean() > 0:
                # Mean movie rating if there is no similar for the computation
                return self.df.rating [self.df['movieId'] ==
                movie_id]. mean()
            else :
            # else mean user rating
                return self.df.rating [self.df['userId'] ==
            user_id]. mean()
        return rating_num/rating_den

In [71]:
db=CollaborativeFiltering(ratingsdf)
db.sim=f

In [67]:
f=db.sim

In [85]:
db.predict(2,2)

3.674336737207032

In [88]:
import json
json.dump(f,open('data.json','w'))